In [94]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import matplotlib.pyplot as plt

from unidecode import unidecode
from ftfy import fix_encoding
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import pymongo
import pymysql.cursors  

from bson import ObjectId

In [3]:
!pip install pymysql


In [95]:
connection = pymysql.connect(--------)


In [96]:
def query_from_db(query):
    mycursor = connection.cursor()
    mycursor.execute(query)

    qs = []
    current = None

    for x in mycursor:
        code = x[0]
        content = fix_encoding(x[1])
        dapan = fix_encoding(x[2])
        cd = x[3]
        if current is None:
            current = {
                'code': code,
                'content': content,
                'answers': [dapan],
                'cd': cd
            }
        elif code == current['code']:
            if dapan != 'Em không làm được' and dapan != 'Đáp án khác':
                current['answers'].append(dapan)
        else:
            qs.append(current)
            current = {
                'code': code,
                'content': content,
                'answers': [dapan],
                'cd': cd
            }
    return qs

In [97]:
questions = []
questions.extend(query_from_db("SELECT c.maso,c.content,d.content,o.maso AS cd FROM cau_hoi AS c INNER JOIN chuyen_de_con AS o ON o.ID_CD=c.ID_CD INNER JOIN dap_an_ngan AS d ON d.ID_C = c.ID_C AND d.content != '.' WHERE c.ID_MON = 1 AND c.ID_CD != 0 ORDER BY c.ID_C DESC"))
questions.extend(query_from_db("SELECT c.maso,c.content,d.content,CONCAT('00', SUBSTRING_INDEX(c.maso, '-', 1)) AS cd FROM cau_hoi AS c INNER JOIN dap_an_ngan AS d ON d.ID_C = c.ID_C AND d.content != '.' WHERE c.ID_CD = 0 AND c.ID_MON = 1 AND (c.maso LIKE 'D%' OR c.maso LIKE 'H%') ORDER BY c.ID_C DESC"))

In [28]:
# subject = db['subjects'].find_one({'_id': ObjectId("5c31bcdf97ad515dbfa6cb4a")}, {'questions': 1})

In [98]:
# questions = subject['questions']
len(questions)

12902

In [99]:
questions[0]

{'code': 'LY-02-55ba40',
 'content': '  Hai chất điểm M và N dao động điều hòa dọc theo hai đường thẳng song song và coi như ở sát với nhau và coi như cùng gốc tọa độ O. Phương trình dao động của chúng lần lượt là   \\({x_1} = {A_1}\\cos \\left( {10t + \\frac{\\pi }{3}} \\right)\\,\\,(cm)\\) \xa0và\xa0  \\({x_2} = {A_2}\\cos \\left( {10t - \\frac{\\pi }{6}} \\right)\\,\\,(cm)\\) . Biết rằng  \\(\\frac{{x_1^2}}{{36}} + \\frac{{x_2^2}}{{64}} = 1\\) .\xa0Khi đó vận tốc cực đại của chất điểm thứ hai có độ lớn bằng',
 'answers': ['60 cm/s.',
  '\\(60\\sqrt 2 \\) cm/s.',
  '80 cm/s.',
  '\\(80\\sqrt 2 \\) cm/s.'],
 'cd': 'XXXXX'}

In [100]:
distinct_questions = []
exist = {}
for q in questions:
    if q['content'] in exist:
        continue
    distinct_questions.append(q)
    exist[q['content']] = True

In [101]:
len(distinct_questions)

10597

In [107]:
df = pd.DataFrame(distinct_questions)
df = df.sample(frac=1).reset_index(drop=True)
# df = df.drop(['answer_detail', 'created_at', 'unuse', 'image_right', 'note', 'storage_ids', 'updated_at'], axis=1)
# df['answers'] = df['answers'].apply(lambda x: ' ANSWERS:' + ' | '.join(x[0]['answer_details']))
df['answers'] = df['answers'].apply(lambda x: ' ANSWERS:' + ' | '.join(x))

In [108]:
df.head()

,answers,cd,code,content
0,ANSWERS: \(m \leqslant \frac{2}{3}\) | \(m ...,00D07,00-24a-3a8,Tìm m để hàm số \(y = {x^3} - 2{x^2} - mx + ...
1,ANSWERS: \(\int {{e^{2x}}dx} = 2{e^{2x}} + C....,0000,00-00z-0a38,Tìm nguyên hàm của hàm số \(f(x) = {e^{2x}}.\)
2,ANSWERS: Tam giác | Tứ giác | Ngũ giác | L...,02H01,02-03a-2a14,"Cho hình chóp \(S.ABCD\) có M,N,P lần lượt ..."
3,ANSWERS: \(y' = \frac{{ - 2{{\cot }^3}x}}{{{{...,00D06,D06-02b-3a28,(28) Cho hàm số \(y = \sqrt {{{\cot }^4}x + 1...
4,ANSWERS: \(\frac{1}{6}\ln 6 - 7\ln 3\) | \...,00D09,D09-01c-0a17,Cho \(F\left( x \right)\) là một nguyên hàm ...


In [104]:
df.columns


Index(['answers', 'cd', 'code', 'content'], dtype='object')

In [109]:
new_df = df[(df['cd'] != 'XXX') & (df['cd'] != '0202') & (df['cd'] != '0000') & (df['cd'] != '00D11')]
new_df['cd'] = new_df['cd'].apply(lambda x: x[2:])
new_df.to_csv('question1.csv', index=False)
len(df)

C:\Users\VIET\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


10597

In [117]:
df = new_df[new_df['cd'] != 'XXX']
df['cd'].unique()

array(['D07', 'H01', 'D06', 'D09', 'H04', 'D05', 'H07', 'D01', 'D04',
       'H02', 'D10', 'D03', 'D08', 'H06', 'H03', 'H05', 'D02', 'H08'],
      dtype=object)

In [118]:
import re

def cleaner(text):
    # normalize
    text = unidecode(text)
#     text = re.sub(r'[^A-Za-z ]', ' ', text.strip(' \n\r\t'))
    text = text.lower().strip(' \n\r\t')
    text = text.replace('\\', ' ').strip(' \n\r\t')
    text = text.replace('-', '+')
    text = re.sub(r'[0-9]+', '0', text.strip(' \n\r\t'))
    text = re.sub(r'[\[\]}{()]', ' ', text.strip(' \n\r\t'))
    text = re.sub(r'(?<=[^0-9])\.(?=\d+)', '', text.strip(' \n\r\t'))
    text = re.sub(r'\|<\|0+0a+0+image0\|>\||\\|<\/font>|<font|color="#[0-9]+">|<img src="[^>]+" \/>|<i>|</i>|<br| \/>', '', text.strip(' \n\r\t'))
    text = re.sub(r'[ ]{2,}', ' ', text.strip(' \n\r\t'))
    return text.strip(' \n\r\t')

In [119]:
def split_only(text):
#     parts = re.findall('(?<={).*(?=})', text)
#     return re.split('[\\(|\\)|\{| ]{1,}', text)
#     print(parts)
#     return re.split('[~`!@#$%^&*()\-=+\/><,.;:\'"\]\[{}| ]{1,}', text)
    tokens = text.split(' ')
#     return [token for token in tokens if len(token) > 1]
    return tokens

In [120]:
text = 'Cho phương trình  \\({6^{{x^3} - 1}}{.216^{{x^2}}} = {36^{\\frac{x}{2} + 1}}\\) . Tích các giá trị của x là'
print(cleaner(text))
print(split_only(cleaner(text)))

cho phuong trinh 0^ x^0 + 0 0^ x^0 = 0^ frac x 0 + 0 . tich cac gia tri cua x la
['cho', 'phuong', 'trinh', '0^', 'x^0', '+', '0', '0^', 'x^0', '=', '0^', 'frac', 'x', '0', '+', '0', '.', 'tich', 'cac', 'gia', 'tri', 'cua', 'x', 'la']


In [121]:
df['text_encoded'] = df['content'].apply(lambda s: cleaner(s))
df['answers_encoded'] = df['answers'].apply(lambda s: cleaner(s))
df['text_encoded'] = df['text_encoded'].map(str) + " " + df['answers_encoded']
df_fix = df.drop(['content', 'answers', 'code'], axis=1)
df_fix.head()

C:\Users\VIET\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\VIET\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\VIET\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

,cd,text_encoded,answers_encoded
0,D07,tim m de ham so y = x^0 + 0 x^0 + mx + 0 dong ...,answers: m leqslant frac 0 0 | m in mathbb r |...
2,H01,"cho hinh chop s.abcd co m,n,p lan luot nam tre...",answers: tam giac | tu giac | ngu giac | luc giac
3,D06,0 cho ham so y = sqrt cot ^0 x + 0 . dao ham c...,answers: y' = frac + 0 cot ^0 x sin ^0 x sqrt ...
4,D09,cho f left x right la mot nguyen ham cua f lef...,answers: frac 0 0 ln 0 + 0 ln 0 | 0 ln 0 + fra...
5,D07,cho ham so y = x^0 + 0 x^0 + 0 . tim m de phuo...,answers: m in left 0;0 right | m in left 0;0 r...


In [122]:
df_fix[:1]['text_encoded'].values

array(['tim m de ham so y = x^0 + 0 x^0 + mx + 0 dong bien tren left 0; + infty right answers: m leqslant frac 0 0 | m in mathbb r | m leqslant 0 | m geqslant 0'],
      dtype=object)

In [51]:
df_fix.to_csv('questions.csv', index=False)


In [123]:
df_fix.shape

(9829, 3)

In [124]:
vectorizer = TfidfVectorizer(max_df=0.2,
#                              max_features=200,
                             min_df=10,
                             ngram_range=(2,2),
                             tokenizer=split_only)
features = vectorizer.fit_transform(df['text_encoded'])

In [125]:
print("n_samples: %d, n_features: %d" % features.shape)

n_samples: 9829, n_features: 4935


In [126]:
vectorizer.get_feature_names()

['\n b',
 '\n end',
 '\n left',
 '\n x',
 '* .',
 '* 0',
 '* co',
 '+ ...',
 '+ ....',
 '+ 0,',
 '+ 0,0',
 '+ 0.',
 '+ 0.0',
 '+ 0.0^',
 '+ 0.0^x',
 '+ 0;',
 '+ 0;\n+',
 '+ 0;0',
 '+ 0;0;',
 '+ 0;0;\n+',
 '+ 0;0;0',
 '+ 0^',
 '+ 0^0',
 '+ 0^n',
 '+ 0^x',
 '+ 0a',
 '+ 0b',
 '+ 0b.',
 '+ 0c_',
 '+ 0c_n^0',
 '+ 0g',
 '+ 0i',
 '+ 0i;',
 '+ 0m',
 '+ 0mx',
 '+ 0n',
 '+ 0t',
 '+ 0x',
 '+ 0y',
 '+ 0z',
 '+ a',
 '+ a^0',
 '+ a_',
 '+ a_0',
 '+ ax',
 '+ b',
 '+ b?',
 '+ b^0',
 '+ b_0',
 '+ bi',
 '+ bx',
 '+ c',
 '+ c.',
 '+ c_',
 '+ c_n^0',
 '+ cos',
 '+ cot',
 '+ cx',
 '+ d',
 '+ e^',
 '+ f',
 '+ frac',
 '+ g',
 '+ i',
 '+ i;',
 '+ infty',
 '+ int',
 '+ k',
 '+ k0',
 '+ k_0',
 '+ left',
 '+ left|',
 '+ ln',
 '+ log',
 '+ m',
 '+ m^0',
 '+ mx',
 '+ n',
 '+ n^0',
 '+ operatorname',
 '+ overline',
 '+ overrightarrow',
 '+ pi',
 '+ s_',
 '+ s_0',
 '+ sin',
 '+ sqrt',
 '+ t',
 '+ t^0',
 '+ tan',
 '+ text',
 '+ u_0',
 '+ u_n',
 '+ x',
 '+ x^0',
 '+ x_0',
 '+ x_0^0',
 '+ y',
 '+ y^0',
 '+ z',
 '+ z^0'

In [127]:
from sklearn.cluster import DBSCAN, KMeans, MiniBatchKMeans
from sklearn.metrics import pairwise_distances

In [128]:
clustering = KMeans(n_clusters=19).fit(features)
# clustering = DBSCAN(eps=0.55, min_samples=3).fit(features)

In [129]:
print(len(set(clustering.labels_)))
print(set(clustering.labels_))

19
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}


In [130]:
sorted(list(map(lambda x: (x, len(np.where(clustering.labels_ == x)[0])), list(set(clustering.labels_)))), key=lambda x: x[1], reverse=True)

[(8, 2788),
 (2, 702),
 (7, 684),
 (12, 510),
 (1, 503),
 (5, 472),
 (17, 470),
 (15, 448),
 (4, 444),
 (6, 364),
 (16, 360),
 (9, 355),
 (13, 338),
 (10, 326),
 (11, 310),
 (18, 228),
 (3, 196),
 (0, 175),
 (14, 156)]

In [131]:
df['label'] = clustering.labels_
df

C:\Users\VIET\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,answers,cd,code,content,text_encoded,answers_encoded,label
0,ANSWERS: \(m \leqslant \frac{2}{3}\) | \(m ...,D07,00-24a-3a8,Tìm m để hàm số \(y = {x^3} - 2{x^2} - mx + ...,tim m de ham so y = x^0 + 0 x^0 + mx + 0 dong ...,answers: m leqslant frac 0 0 | m in mathbb r |...,1
2,ANSWERS: Tam giác | Tứ giác | Ngũ giác | L...,H01,02-03a-2a14,"Cho hình chóp \(S.ABCD\) có M,N,P lần lượt ...","cho hinh chop s.abcd co m,n,p lan luot nam tre...",answers: tam giac | tu giac | ngu giac | luc giac,4
3,ANSWERS: \(y' = \frac{{ - 2{{\cot }^3}x}}{{{{...,D06,D06-02b-3a28,(28) Cho hàm số \(y = \sqrt {{{\cot }^4}x + 1...,0 cho ham so y = sqrt cot ^0 x + 0 . dao ham c...,answers: y' = frac + 0 cot ^0 x sin ^0 x sqrt ...,17
4,ANSWERS: \(\frac{1}{6}\ln 6 - 7\ln 3\) | \...,D09,D09-01c-0a17,Cho \(F\left( x \right)\) là một nguyên hàm ...,cho f left x right la mot nguyen ham cua f lef...,answers: frac 0 0 ln 0 + 0 ln 0 | 0 ln 0 + fra...,15
5,ANSWERS: \(m \in \left[ {0;1} \right]\) | \...,D07,00-18a-4a8,Cho hàm số \(y = {x^4} - 4{x^2} + 3\) . Tìm ...,cho ham so y = x^0 + 0 x^0 + 0 . tim m de phuo...,answers: m in left 0;0 right | m in left 0;0 r...,5
6,ANSWERS: \(\left( {4;0} \right)\) | \(\left...,D07,00-18a-1a1,Đồ thị hàm số đi qua điểm nào dưới đây ?,do thi ham so di qua diem nao duoi day ? answe...,answers: left 0;0 right | left 0;0 right | lef...,5
7,ANSWERS: \(\frac{a}{{\sqrt 6 }}\) | \(a\sqr...,H04,00-20b-3a7,Cho hình chóp đều S.ABCD.Gọi O là tâm đáy.Cạn...,cho hinh chop deu s.abcd.goi o la tam day.canh...,answers: frac a sqrt 0 | a sqrt 0 | a frac sqr...,2
8,ANSWERS: \( + \infty \) | 0 | \(\frac{1}{{...,D05,D05-05b-0a8,Tình giới hạn: \(\lim \left( {\sqrt {a{n^2} +...,tinh gioi han: lim left sqrt a n^0 + 0 + sqrt ...,answers: + infty | 0 | frac 0 0 + sqrt a | sqr...,8
9,ANSWERS: \(13\ln 2 + 6\left( {m/s} \right)\) ...,D09,10-50a-2001a47,Một vật chuyển động với vận tốc \(v\left( t ...,mot vat chuyen dong voi van toc v left t right...,answers: 0 ln 0 + 0 left m/s right | 0 left m/...,8
10,ANSWERS: \(V = \pi \int\limits_0^1 {\sqrt {2x...,D09,01-73a-2a4,Cho hình phẳng \(\left( D \right)\) được giớ...,cho hinh phang left d right duoc gioi han boi ...,answers: v = pi int limits_0^0 sqrt 0x + 0 tex...,8


In [132]:
query = 2
limit = 20

# df[df['label'] == 30].head(10)[['_id', 'code', 'text_encoded', 'level', 'label']].values
df_filtered = df[df['label'] == query]
print(len(df_filtered))
# df_filtered.head(10)[['code', 'content', 'answers', 'label']].values
df_filtered.head(10)[['code','text_encoded', 'label']].values

702


array([['00-20b-3a7',
        'cho hinh chop deu s.abcd.goi o la tam day.canh day = so = a.tinh khoang cach tu o den mat phang sbc theo a answers: frac a sqrt 0 | a sqrt 0 | a frac sqrt 0 0 | frac a sqrt 0',
        2],
       ['02-26a-4a13',
        "cho lang tru deu abc.a'b'c' co ab = aa' = a .tinh khoang cach tu a den mat phang a'bc theo a answers: frac a sqrt 0 0 | frac a sqrt 0 | frac a sqrt 0 | frac a 0",
        2],
       ['00-35a-2a3',
        'mot hinh hop hinh chu nhat noi tiep mat cau va co ba kich thuoc la a , b , c . tinh ban kinh cua mat cau. answers: sqrt a^0 + b^0 + c^0 | sqrt 0 left a^0 + b^0 + c^0 right | frac sqrt a^0 + b^0 + c^0 0 | frac 0 0 sqrt a^0 + b^0 + c^0',
        2],
       ['00-40a-1a38',
        "cho hinh lang tru dung abc.a'b'c' co ab = 0 , ac = 0 , aa' = 0 va widehat bac = 0^ circ . goi m , n lan luot la cac diem tren canh bb' , cc' sao cho bm = 0b'm ; cn = 0c'n . tinh khoang cach tu diem m den mat phang left a'bn right . answers: frac 0 sqrt 0 0 | fra

In [307]:
from sklearn.metrics import 
 

In [308]:
for i in range(20, 200, 20):
    clustering = MiniBatchKMeans(n_clusters=i).fit(features)
    silhouette_avg = silhouette_score(features, clustering.labels_)
    print("For n_clusters =", i,
          "The average silhouette_score is :", silhouette_avg)

For n_clusters = 20 The average silhouette_score is : 0.03701320697681477
For n_clusters = 40 The average silhouette_score is : 0.05290065453947901
For n_clusters = 60 The average silhouette_score is : 0.02838074300891473
For n_clusters = 80 The average silhouette_score is : 0.027800731606158622
For n_clusters = 100 The average silhouette_score is : 0.023823939084306703
For n_clusters = 120 The average silhouette_score is : 0.020909992085143787
For n_clusters = 140 The average silhouette_score is : 0.024134519498162647
For n_clusters = 160 The average silhouette_score is : 0.035515488919295714
For n_clusters = 180 The average silhouette_score is : 0.03671282495566707
